In [10]:
import pandas as pd
import numpy as np
from  matplotlib import pyplot as plt
import networkx as nx

In [18]:
#def a function to filter the original dataset and create the RoW node
def filt_Z(dfz_rough):
    # Eliminate columns and rows that start with specific prefixes
    prefixes = ("XAM", "XEU", "XAF", "XAS")

    # Filter columns that do not start with the specified prefixes
    dfz_total = dfz_rough.loc[~dfz_rough.index.str.startswith(prefixes), ~dfz_rough.columns.str.startswith(prefixes)] 

    countries_to_keep = ['AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHN', 'CYP', 'CZE', 'DEU', 'DNK', 'ESP', 
                     'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 
                     'LTU', 'LUX', 'LVA', 'MEX', 'MLT', 'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SVK', 'SVN', 
                     'SWE', 'TUR', 'USA']
    # Filter rows and columns that start with any of the country codes in countries_to_keep
    dfz_base = dfz_total.loc[dfz_total.index.str.startswith(tuple(countries_to_keep)), dfz_total.columns.str.startswith(tuple(countries_to_keep))]

    new_labels_rows_columns = ["RoW_AGRagr", "RoW_AGRfor", "RoW_AGRfis", "RoW_ENRcoa", "RoW_ENRoil", "RoW_ENRgas", "RoW_MIN+", "RoW_MANfoo",
"RoW_MANtex", "RoW_MANwoo", "RoW_MANpap", "RoW_MANpri", "RoW_MANche", "RoW_MANpha",
"RoW_MANpla", "RoW_MANmin", "RoW_MANmet", "RoW_MANfmp", "RoW_MANmac", "RoW_MANmot",
"RoW_MANtra", "RoW_MANrep", "RoW_MANcom", "RoW_MANele", "RoW_MANfur", "RoW_ENRele",
"RoW_WATwat", "RoW_WATwst", "RoW_CNS+", "RoW_TRDmot", "RoW_TRAinl",
"RoW_TRAair", "RoW_TRAwar", "RoW_TRApos", "RoW_FD+", "RoW_COMpub", "RoW_COMvid",
"RoW_COMcom", "RoW_FINser", "RoW_RES+", "RoW_PROleg", "RoW_ADM+", "RoW_PUB+",
"RoW_EDU+", "RoW_HEA+", "RoW_ART+", "RoW_EXT+"]

    # List of country codes contained in RoW
    countries_of_RoW = ["AFG", "AGO", "ALB", "ARE", "ARG", "ARM", "AZE", "BDI", "BEN", "BFA", "BGD", "BHR", "BHS", "BIH", "BLR", "BLZ", "BOL", 
"BRN", "BTN", "BWA", "CAF", "CIV", "CMR", "COD", "COG", "COL", "CRI", "CUB", "DJI", "DYE", "DOM", "DZA", "ECU", "EGY", 
"ERI", "ETH", "GAB", "GEO", "GHA", "GIN", "GMB", "GNQ", "GTM", "HND", "HKG", "HTI", "IRQ", "ISL", "ISR", "JAM", "JOR", 
"KAZ", "KEN", "KGZ", "KHM", "KWT", "LAO", "LBN", "LBR", "LBY", "LKA", "MDA", "MDG", "MKD", "MLI", "MMR", "MNG", "MOZ", 
"MRT", "MWI", "MYS", "NAM", "NER", "NGA", "NIC", "NPL", "NZL", "OMN", "PAK", "PSE", "PAN", "PER", "PHL", "PNG", "PRK", 
"PRY", "QAT", "RWA", "SAU", "SDS", "SEN", "SGP", "SLE", "SLV", "SOM", "SRB", "SDN", "SYR", "TCD", "TGO", "THA", "TJK", 
"TKM", "TUN", "TZA", "UGA", "UKR", "URY", "UZB", "VEN", "VNM", "YEM", "ZAF", "ZMB", "ZWE"
]

    #create a new data frame for Row industries as suppliers
    new_data = []
    
    for i in new_labels_rows_columns:
        for j in dfz_base.index: 
            new_value = 0
             #replace the country_sector index i with a country-sector index of a country of the RoW
            for n in countries_of_RoW:
                # Replace "RoW_" with the current country from countries_of_RoW followed by "_"
                i_replaced = i.replace("RoW_", n + "_")
                new_value =  new_value + dfz_total.loc[i_replaced, j]
            new_data.append({'i': i, 'j': j, 'value': new_value})
                
    # Convert the collected data to a DataFrame (assuming 'i' and 'j' are new index and column)
    new_data_df = pd.DataFrame(new_data)
    matrix_df = new_data_df.pivot(index='i', columns='j', values='value')
    
    #glue matrix df to dfz_base from below
    dfz_base_2=pd.concat([dfz_base, matrix_df]) 

    #create a new data frame for Row industries as buyers
    new_data2 = []
    for i in dfz_base.index:
        for j in new_labels_rows_columns:
            new_value2 = 0
            for n in countries_of_RoW:
                j_replaced=j.replace("RoW", n)
                new_value2 += dfz_total.loc[i, j_replaced]
            new_data2.append({'i': i, 'j': j, 'value': new_value2})
        
    # Convert the collected data to a DataFrame (assuming 'i' and 'j' are new index and column)
    new_data_df2 = pd.DataFrame(new_data2)
    matrix_df2 = new_data_df2.pivot(index='i', columns='j', values='value')

    #create a new data frame for Row industries as suppliers and buyers
    new_data3 = []
    for i in new_labels_rows_columns:
        for j in new_labels_rows_columns:
            new_value3 = 0
            for m in countries_of_RoW:
                for n in countries_of_RoW:
                    i_replaced=i.replace("RoW", m)
                    j_replaced=j.replace("RoW", n)
                    new_value3 += dfz_total.loc[i_replaced, j_replaced]
            #new_value3 = new_value3/ X_Row.loc[j, 'X_(RoW, sec)']
            new_data3.append({'i': i, 'j': j, 'value': new_value3})
    
    # Convert the collected data to a DataFrame (assuming 'i' and 'j' are new index and column)
    new_data_df3 = pd.DataFrame(new_data3)

    matrix_df3 = new_data_df3.pivot(index='i', columns='j', values='value')
    matrix_df23=pd.concat([matrix_df2, matrix_df3])
    dfz = pd.concat([dfz_base_2, matrix_df23], axis=1)
    return dfz
            
            
#def a function to create the dataframe for the countries
def Z_countries(dfz):
    
    sectors = ['AGRagr', 'AGRfor', 'AGRfis', 'ENRcoa', 'ENRoil', 'ENRgas', 'MIN+', 'MANfoo', 'MANtex', 'MANwoo',
    'MANpap', 'MANpri', 'MANche', 'MANpha', 'MANpla', 'MANmin', 'MANmet', 'MANfmp',
    'MANmac', 'MANmot', 'MANtra', 'MANrep', 'MANcom', 'MANele', 'MANfur', 'ENRele',
    'WATwat', 'WATwst', 'CNS+', 'TRDmot', 'TRAinl', 'TRAair', 'TRAwar', 'TRApos',
    'FD+', 'COMpub', 'COMvid', 'COMcom', 'FINser', 'RES+', 'PROleg', 'ADM+', 'PUB+',
    'EDU+', 'HEA+', 'ART+', 'EXT+']
    countries = ['AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHN', 'CYP', 'CZE', 'DEU', 'DNK', 'ESP', 
                     'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN', 'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 
                     'LTU', 'LUX', 'LVA', 'MEX', 'MLT', 'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SVK', 'SVN', 
                     'SWE', 'TUR', 'USA', 'RoW']
    dfz_c = pd.DataFrame(0.0, index=countries, columns=countries)

    for country1 in countries:
        for country2 in countries:
            for sector1 in sectors:
                for sector2 in sectors:
                     # Construct the row and column keys
                    row_key = f"{country1}_{sector1}"
                    col_key = f"{country2}_{sector2}"
                
                    # Check if keys exist in df_A
                    if row_key in dfz.index and col_key in dfz.columns:
                        dfz_c.loc[country1, country2] += dfz.loc[row_key, col_key]
                    else:
                        print(f"Missing key: {row_key} or {col_key} not found in df_A")
    return dfz_c
                


In [20]:
# Define the years and custom labels
years = range(2010, 2021)
labels = ['eu', 'gl', 'bc']
all_identifiers = list(years) + labels # Combine years and labels into a single iterable

# Loop through all identifiers
for identifier in all_identifiers:
    # Read the CSV file for the given identifier
    dfz_rough = pd.read_csv(f'Z_matrix_{identifier}.csv', sep=',')
    dfz_rough.columns = [k.replace(' ', '') for k in dfz_rough.columns]
    dfz_rough.set_index("Unnamed:0", inplace=True)
    
    # Apply the transformations
    dfz = filt_Z(dfz_rough)
    dfz_c = Z_countries(dfz) 
    print(f"Info dfz {identifier} : dfz.info()")
    print(f"Info dfz_c {identifier} : dfz_c.info()")
    
    # Save the results as parquet files
    dfz.to_parquet(f'Complex-Network/dfz_{identifier}.parquet', engine='pyarrow')
    dfz_c.to_parquet(f'Complex-Network/dfz_{identifier}_c.parquet', engine='pyarrow')
